# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our loss function.

Sometimes you need to get or manipulate parameters.

In this section, we implement the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Parameter initialization.
* Sharing parameters across different model components.

(**We start by focusing on an MLP with one hidden layer.**)


In [1]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-1.3509198],
       [-0.5090279]], dtype=float32)>

## [**Parameter Access**]

How to access parameters from the models that you already know?

When a model is defined via the `Sequential` class,
we can first access any layer by indexing
into the model as though it were a list.

We can inspect the parameters of the second fully-connected layer as follows.


In [2]:
print(net.layers[2].weights)

[<tf.Variable 'dense_1/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.39221483],
       [-0.19289303],
       [-0.93515116],
       [-0.24210244]], dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


The output shows that this is a fully-connected layer
containing two parameters,
corresponding to that layer's
weights and biases, respectively. Both are stored as single precision floats (float32).

As you can see the names of the parameters allow us to uniquely identify
each layer's parameters, even in a network containing hundreds of layers.

The following code extracts the bias
from the second neural network layer, which returns a parameter class instance, and 
further accesses that parameter's value.

In [3]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


### [**All Parameters at Once**]

Sometimes we need to perform operations on all parameters instead of
accessing them one-by-one can be a tedious task.

Below we demonstrate accessing the parameters of the first fully-connected layer vs. accessing all layers.


In [4]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.27596825,  0.39735395,  0.22277027, -0.37036708],
       [ 0.5029089 , -0.68404275,  0.5518392 , -0.34678066],
       [ 0.10227382, -0.5717257 ,  0.74360794,  0.15284544],
       [-0.4585268 ,  0.24101466,  0.05636376,  0.12352437]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.27596825,  0.39735395,  0.22277027, -0.37036708],
       [ 0.5029089 , -0.68404275,  0.5518392 , -0.34678066],
       [ 0.10227382, -0.5717257 ,  0.74360794,  0.15284544],
       [-0.4585268 ,  0.24101466,  0.05636376,  0.12352437]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[-0.39221483],
       [-0.19289303],
       [-0.93515116],
       [-0.24210244]], dtype=float32), array([0.], dtype=float32)]


This provides us with another way of accessing the parameters of the network as follows.


In [5]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

### [**Collecting Parameters from Nested Blocks**]

How the parameter naming conventions work if we nest multiple blocks inside each other?

First we  define a function that produces blocks and then
combine these inside yet larger blocks.


In [6]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)], name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.10602788],
       [-0.0068799 ]], dtype=float32)>

Now that [**we have designed the network,
let us see how it is organized.**]


In [7]:
print(rgnet.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (2, 4)                    80        
                                                                 
 dense_6 (Dense)             (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


Since the layers are hierarchically nested,
we can also access them as though
indexing through nested lists.

In [8]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_3/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

This code accesses the first major block,
within it the second sub-block,
and within that the bias of the first layer,
with as follows.

## Parameter Initialization

Many deep learning frameworks provide default random initializations to its layers.


How to initialize parameters manually and properly?



TensorFlow provides a variety of initialization methods both in the root module and the `keras.initializers` module.

By default, Keras initializes weight matrices uniformly by drawing from a range that is computed according to the input and output dimension, and the bias parameters are all set to zero.

### [**Built-in Initialization**]

The code below calls a built-in initializer and initializes all weight parameters
as Gaussian random variables
with standard deviation 0.01, while bias parameters cleared to zero.


In [9]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_7/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.01076516,  0.00603879,  0.00759928, -0.00946514],
        [ 0.009541  , -0.01180921,  0.00209968,  0.0023306 ],
        [ 0.0002064 , -0.01361557, -0.00058213, -0.01754736],
        [ 0.00446019, -0.00530258, -0.00520741,  0.02508139]],
       dtype=float32)>,
 <tf.Variable 'dense_7/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

We can also initialize all the parameters
to a given constant value (say, 1).


In [10]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

[**We can also apply different initializers for certain blocks.**]

The code below initializes the first layer
with the Xavier initializer
and initialize the second layer
to a constant value of 42.


In [11]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.4876433 , -0.641036  ,  0.09287953,  0.3265683 ],
       [-0.50385475, -0.44116542,  0.43946177, -0.0542168 ],
       [-0.2645722 , -0.8347696 ,  0.8092012 ,  0.36214978],
       [ 0.64848894, -0.76763314, -0.09440249, -0.47151065]],
      dtype=float32)>
<tf.Variable 'dense_12/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


### [**Custom Initialization**]

Sometimes, the initialization methods we need
are not provided by the deep learning framework.

In the following example we implement an initializer for any weight parameter $w$ using the following strange distribution:

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


Here we define a subclass of `Initializer` and implement the `__call__`
function that return a desired tensor given the shape and data type.


In [12]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data = tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor = (tf.abs(data) >= 5)
        factor = tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu,
                          kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 5.531039 ,  0.       ,  0.       ,  6.350502 ],
       [ 0.       , -0.       , -5.0844812,  6.5772667],
       [ 7.0711613, -7.905321 , -7.165332 ,  5.0007486],
       [ 0.       ,  0.       ,  7.868946 ,  0.       ]], dtype=float32)>


Note that we always have the option
of setting parameters directly.


In [13]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  1.       ,  1.       ,  7.350502 ],
       [ 1.       ,  1.       , -4.0844812,  7.5772667],
       [ 8.071161 , -6.905321 , -6.165332 ,  6.0007486],
       [ 1.       ,  1.       ,  8.868946 ,  1.       ]], dtype=float32)>

## [**Tied Parameters**]

How to share parameters across multiple layers?

In the following code we allocate a dense layer
and then use its parameters specifically
to set those of another layer.


In [14]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(), shared, shared,
    tf.keras.layers.Dense(1),])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


## Summary

* We have several ways to access, initialize, and tie model parameters.
* We can use custom initialization.


## Exercises (Optional)

1. Use the `FancyMLP` model defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Look at the initialization module document to explore different initializers.
1. Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?
